In [1]:
import pandas as pd
import sys
import numpy as np

## Orders


In [2]:
import src.order

In [3]:
order_type = "bid"

qty_b1 = 100
price_b1 = 99
bid1 = src.order.Order(order_type, qty_b1, price_b1)

qty_b2 = 10
price_b2 = 98
bid2 = src.order.Order(order_type, qty_b2, price_b2)

order_type = "ask"

qty_a1 = 100
price_a1 = 101
ask1 = src.order.Order(order_type, qty_a1, price_a1)

qty_a2 = 10
price_a2 = 102
ask2 = src.order.Order(order_type, qty_a2, price_a2)

## Order Book


In [4]:
from src.order_book import OrderBook

orderbook = OrderBook([bid1, bid2], [ask1, ask2])
print(orderbook)

┌─────────┬────────┬─────────┬─────────┐
│   Index │   Size │   Price │   Total │
├─────────┼────────┼─────────┼─────────┤
│       0 │     10 │     102 │    1020 │
│       1 │    100 │     101 │   10100 │
│       2 │    100 │      99 │    9900 │
│       3 │     10 │      98 │     980 │
└─────────┴────────┴─────────┴─────────┘


## Main Script


In [5]:
from src.kandel import kandel_simulator
from src.time_series import load_csv

In [6]:
ts = load_csv("data/ETHUSDC-1s-2024-01>08.csv")[:4*30*24*3600]

In [7]:
historical_vol = ts.to_pandas()["price"].rolling(3600 * 24).std()
hv_1h = historical_vol.resample("1h").last()
implied_vol = pd.read_csv(
    "data/ETHBVOLUSDT-2024-01>05.csv", index_col=0, parse_dates=True
)
implied_vol.index = pd.to_datetime(implied_vol.index, unit="s")
implied_vol = implied_vol.align(historical_vol, join="inner", axis=0)[0]

/var/folders/kn/wgl1sqns1bxgynxc599_5y9c0000gp/T/ipykernel_37519/1452361257.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  implied_vol = pd.read_csv(


In [8]:
window = 3600 * 24
quote = 37500  # USDC
base = 37500  # also in USDC, will be swapped to base at day 0
vol_mult = 1.645
n_points = 10
step_size = 1
days = 11

transactions, res, order_book_history = kandel_simulator(
    ts=ts[-(days + 1) * 24 * 3600 :],
    quote=quote,
    base=base,
    vol_mult=vol_mult,
    n_points=n_points,
    step_size=step_size,
    window=window,
    historical_vol=historical_vol[-(days + 1) * 24 * 3600 :],
    implied_vol=implied_vol["bvol"][-(days + 1) * 24 * 3600 :],
)
res.to_pandas().to_csv("results/simul_results.csv")

100%|██████████| 950400/950400 [00:15<00:00, 62645.92it/s] 


# Results


In [9]:
import pandas as pd  # type: ignore

pd.options.plotting.backend = "plotly"

from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import datetime

In [10]:
res = pd.read_csv("results/simul_results.csv", index_col=0, parse_dates=True)

In [11]:
res["mtm_eth"] = res["mtm"] / res["price"]
res["base_returns"] = res["mtm_eth"] / ((quote + base) / res["price"][window]) - 1
res["quote_returns"] = res["mtm"] / (quote + base) - 1

/var/folders/kn/wgl1sqns1bxgynxc599_5y9c0000gp/T/ipykernel_37519/4188196617.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  res["base_returns"] = res["mtm_eth"] / ((quote + base) / res["price"][window]) - 1


In [12]:
window_hours = window // 3600 - 1
res_1h = res.resample("1h").last()
iv_1h = implied_vol.resample("1h").last()
iv_1h[-len(res_1h) :]

# PLOT
fig = make_subplots(
    rows=3,
    cols=1,
    shared_xaxes=True,
    specs=[[{"secondary_y": True}], [{}], [{"secondary_y": True}]],
    vertical_spacing=0.05,
    subplot_titles=("MTM", "Returns", "Price"),
)
fig.add_trace(
    go.Scatter(
        x=res_1h.index[window_hours:],
        y=res_1h.mtm[window_hours:],
        mode="lines",
        name="MTM in USDC",
    ),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=res_1h.index[window_hours:],
        y=res_1h.mtm_eth[window_hours:],
        mode="lines",
        name="MTM in ETH",
    ),
    row=1,
    col=1,
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(
        x=res_1h.index[window_hours:],
        y=res_1h.quote_returns[window_hours:],
        mode="lines",
        name="Over holding USDC",
    ),
    row=2,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=res_1h.index[window_hours:],
        y=res_1h.base_returns[window_hours:],
        mode="lines",
        name="Over holding ETH",
    ),
    row=2,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=res_1h.index[window_hours:],
        y=res_1h.price[window_hours:],
        mode="lines",
        name="Price",
    ),
    row=3,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=iv_1h.index[-len(res_1h) + window_hours :],
        y=iv_1h.bvol[-len(res_1h) + window_hours :],
        mode="lines",
        name="IV"
    ),
    secondary_y=True,
    row=3,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=hv_1h.index[-len(res_1h)+ window_hours :],
        y=hv_1h.values[-len(res_1h) + window_hours:],
        mode="lines",
        name="HV"
    ),
    secondary_y=True,
    row=3,
    col=1,
)

fig.update_layout(
    height=1000,
    width=1400,
    title_text=f"MTM, Returns and Price.<br><sup>window = {window} | quote = {quote} | base = {base} | vol_mult = {vol_mult} | n_points = {n_points} | step_size = {step_size} | days = {days}</sup>",
)

fig.update_yaxes(row=2, col=1, tickformat=".2%")

fig.show()

fig.write_image(
    f"results/saves/{window}_{quote}_{base}_{vol_mult}_{n_points}_{step_size}_{days}.png",
    width=1400,
    height=1000,
)

In [13]:
order_book_history_df = pd.DataFrame.from_records(
    [ob.to_dict() for ob in order_book_history]
)
order_book_history_df.index = res[window -1:].index
order_book_history_1h = order_book_history_df.resample("1h").last()

In [14]:
order_book_history_1h

,bids,asks
time,,
2024-04-18 23:00:00,"[3048.717896, 3031.869421, 3015.114058, 2998.4...","[3082.696253, 3099.827179, 3117.053304, 3134.3..."
2024-04-19 00:00:00,"[3031.869421, 3015.114058, 2998.451291, 2981.8...","[3065.66, 3082.696253, 3099.827179, 3117.05330..."
2024-04-19 01:00:00,"[2932.716005, 2916.508604, 2900.390772]","[2965.401506, 2981.88061, 2998.451291, 3015.11..."
2024-04-19 02:00:00,[],[]
2024-04-19 03:00:00,[],[]
...,...,...
2024-04-29 19:00:00,[],[]
2024-04-29 20:00:00,[],[]
2024-04-29 21:00:00,[],[]


In [15]:
fig = make_subplots(
    rows=1,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    subplot_titles=(""),
    specs=[[{"secondary_y": True}]],
)

for i, ob in enumerate(order_book_history_1h.iterrows()):
    bids = ob[1].bids
    asks = ob[1].asks

    fig.add_trace(
        go.Scatter(
            x=[i for _ in range(len(bids))],
            y=tuple(bids),
            mode="markers",
            name="Bids",
            showlegend=False,
            marker=dict(
                color="green",
                symbol="line-ew-open",
                size=8,
                line=dict(width=5, color="black"),
            ),
        ),
    )

    fig.add_trace(
        go.Scatter(
            x=[i for _ in range(len(asks))],
            y=tuple(asks),
            mode="markers",
            name="Asks",
            showlegend=False,
            marker=dict(
                color="red",
                symbol="line-ew-open",
                size=8,
                line=dict(width=5, color="black"),
            ),
        ),
    )

fig.add_trace(
    go.Scatter(
        x=[i for i in range(len(order_book_history_1h))],
        y=res_1h.price[window_hours:],
        mode="lines",
        name="Price",
        marker=dict(color="blue"),
    )
)

fig.update_layout(
    height=1000,
    width=1600,
    title_text=f"Order book<br><sup>window = {window} | quote = {quote} | base = {base} | vol_mult = {vol_mult} | n_points = {n_points} | step_size = {step_size} | days = {days}</sup>",
)

fig.show()

In [16]:
generated_fees = []

for i, t in enumerate(transactions):
    if t:
        generated_fees.append(order_book_history[i+1].to_pandas()["Size"].sum() - order_book_history[i].to_pandas()["Size"].sum())
    else:
        generated_fees.append(0)


   

In [17]:
order_book_history[1]

┌─────────┬─────────┬─────────┬─────────┐
│   Index │    Size │   Price │   Total │
├─────────┼─────────┼─────────┼─────────┤
│       0 │ 1.22323 │ 3240.35 │ 3963.68 │
│       1 │ 1.22323 │ 3222.44 │ 3941.78 │
│       2 │ 1.22323 │ 3204.63 │ 3919.99 │
│       3 │ 1.22323 │ 3186.92 │ 3898.33 │
│       4 │ 1.22323 │ 3169.31 │ 3876.79 │
│       5 │ 1.22323 │ 3151.79 │ 3855.36 │
│       6 │ 1.22323 │ 3134.38 │ 3834.06 │
│       7 │ 1.22323 │ 3117.05 │ 3812.87 │
│       8 │ 1.22323 │ 3099.83 │ 3791.8  │
│       9 │ 1.22323 │ 3082.7  │ 3770.84 │
│      10 │ 1.23002 │ 3048.72 │ 3750    │
│      11 │ 1.23686 │ 3031.87 │ 3750    │
│      12 │ 1.24373 │ 3015.11 │ 3750    │
│      13 │ 1.25065 │ 2998.45 │ 3750    │
│      14 │ 1.2576  │ 2981.88 │ 3750    │
│      15 │ 1.26458 │ 2965.4  │ 3750    │
│      16 │ 1.27161 │ 2949.01 │ 3750    │
│      17 │ 1.27868 │ 2932.72 │ 3750    │
│      18 │ 1.28578 │ 2916.51 │ 3750    │
│      19 │ 1.29293 │ 2900.39 │ 3750    │
└─────────┴─────────┴─────────┴───

In [18]:
order_book_history[72*3600+4]

┌─────────┬─────────┬─────────┬─────────┐
│   Index │    Size │   Price │   Total │
├─────────┼─────────┼─────────┼─────────┤
│       0 │ 1.22805 │ 3266.92 │ 4011.94 │
│       1 │ 1.22805 │ 3254.81 │ 3997.08 │
│       2 │ 1.22805 │ 3242.75 │ 3982.27 │
│       3 │ 1.22805 │ 3230.74 │ 3967.51 │
│       4 │ 1.22805 │ 3218.77 │ 3952.81 │
│       5 │ 1.22805 │ 3206.84 │ 3938.17 │
│       6 │ 1.22805 │ 3194.96 │ 3923.58 │
│       7 │ 1.22805 │ 3183.12 │ 3909.04 │
│       8 │ 1.22805 │ 3171.33 │ 3894.55 │
│       9 │ 1.22805 │ 3159.58 │ 3880.12 │
│      10 │ 1.23262 │ 3136.21 │ 3865.75 │
│      11 │ 1.2372  │ 3124.59 │ 3865.75 │
│      12 │ 1.24181 │ 3113.01 │ 3865.75 │
│      13 │ 1.24642 │ 3101.47 │ 3865.75 │
│      14 │ 1.25106 │ 3089.98 │ 3865.75 │
│      15 │ 1.25571 │ 3078.53 │ 3865.75 │
│      16 │ 1.26038 │ 3067.13 │ 3865.75 │
│      17 │ 1.26507 │ 3055.76 │ 3865.75 │
│      18 │ 1.26977 │ 3044.44 │ 3865.75 │
│      19 │ 1.27449 │ 3033.16 │ 3865.75 │
└─────────┴─────────┴─────────┴───

In [19]:
80 * res["price"][-1]

/var/folders/kn/wgl1sqns1bxgynxc599_5y9c0000gp/T/ipykernel_37519/3542272828.py:1: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



257248.80000000002

In [20]:
generated_fees = pd.Series(generated_fees, index=res.index[window:])
cumulative_fees = generated_fees.cumsum()
cumulative_fees_1h = cumulative_fees.resample("1h").last()
cumulative_fees_1h.plot()

/opt/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [21]:
cumulative_vol = res["volume"].cumsum()
cumulative_vol_1h = cumulative_vol.resample("1h").last()
cumulative_vol_1h.plot()

/opt/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [22]:
cumulative_fees[-1] / res["volume"].sum() * 100

/var/folders/kn/wgl1sqns1bxgynxc599_5y9c0000gp/T/ipykernel_37519/2020928673.py:1: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



5.743082049988855e-05

- Observer quand signal pas trigger mais pertes
- Benchmark du kill switch


- Vérifier la qualité des datas binance
- Tester kill switch plus simple: regarder vol hist plus courte et vol impli séparement 


Idées:
- regarder la corrélation entre return et prix final